<a href="https://colab.research.google.com/github/hwasol/esg/blob/main/%ED%99%98%EA%B2%BD%EA%B0%80%EC%B9%98%EC%A7%80%EC%88%98_%EA%B0%80%EC%A4%91%EC%B9%98%EC%99%80_%EA%B2%B0%EA%B3%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from google.colab import files
uploaded = files.upload()


Saving ND-GAIN 적응 지수 2011.csv to ND-GAIN 적응 지수 2011 (1).csv
Saving 아프리카 재생에너지 비율 2011.csv to 아프리카 재생에너지 비율 2011 (1).csv
Saving 아프리카_CO2_배출량_2010^F2022 (1).csv to 아프리카_CO2_배출량_2010^F2022 (1) (1).csv
Saving 아프리카_CO2_배출량_변화율_2011.csv to 아프리카_CO2_배출량_변화율_2011 (1).csv
Saving 아프리카24개국_보호구역비율_2011.csv to 아프리카24개국_보호구역비율_2011 (1).csv
Saving 아프리카24개국_산림면적_2011.csv to 아프리카24개국_산림면적_2011 (1).csv
Saving 아프리카24개국_환경규제강도_2011.csv to 아프리카24개국_환경규제강도_2011 (1).csv


In [5]:
import pandas as pd

print(uploaded.keys())


dict_keys(['ND-GAIN 적응 지수 2011 (1).csv', '아프리카 재생에너지 비율 2011 (1).csv', '아프리카_CO2_배출량_2010^F2022 (1) (1).csv', '아프리카_CO2_배출량_변화율_2011 (1).csv', '아프리카24개국_보호구역비율_2011 (1).csv', '아프리카24개국_산림면적_2011 (1).csv', '아프리카24개국_환경규제강도_2011 (1).csv'])


In [6]:
co2 = pd.read_csv('아프리카_CO2_배출량_2010^F2022 (1).csv')
co2_rate = pd.read_csv('아프리카_CO2_배출량_변화율_2011.csv')
protected = pd.read_csv('아프리카24개국_보호구역비율_2011.csv')
forest = pd.read_csv('아프리카24개국_산림면적_2011.csv')
regulation = pd.read_csv('아프리카24개국_환경규제강도_2011.csv')
gain = pd.read_csv('ND-GAIN 적응 지수 2011.csv')
renewable = pd.read_csv('아프리카 재생에너지 비율 2011.csv')


In [8]:
co2 = co2[['국가', '2011']]
co2_rate = co2_rate[['국가', '2011']]
protected = protected[['국가', '2011']]
forest = forest[['국가', '2011']]
regulation = regulation[['국가', '2011']]
gain = gain[['국가', '2011']]
renewable = renewable[['국가', '2011']]

co2.columns = ['국가', 'CO2배출량']
co2_rate.columns = ['국가', 'CO2증감률']
protected.columns = ['국가', '보호구역비율']
forest.columns = ['국가', '산림면적']
regulation.columns = ['국가', '환경규제강도']
gain.columns = ['국가', 'NDGAIN지수']
renewable.columns = ['국가', '재생에너지비율']


In [9]:
from functools import reduce

dfs = [co2, co2_rate, protected, forest, regulation, gain, renewable]
df_merged = reduce(lambda left, right: pd.merge(left, right, on='국가', how='inner'), dfs)
df_merged = df_merged.dropna()  # 결측치 행 삭제


In [10]:
from sklearn.preprocessing import StandardScaler

X = df_merged.drop(columns=['국가'])  # 국가명 제외
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [11]:
from sklearn.decomposition import PCA
import numpy as np

pca = PCA()
pca.fit(X_scaled)

# 첫 번째 주성분(PC1) 기준 가중치
weights_pc1 = np.abs(pca.components_[0])
weights_pc1 = weights_pc1 / np.sum(weights_pc1)
weights_df = pd.DataFrame({'지표': X.columns, '가중치(PC1)': weights_pc1})

# 설명력(variance ratio) 반영한 전체 가중치
explained_variance = pca.explained_variance_ratio_
loadings = np.abs(pca.components_)
weighted_loadings = loadings.T @ explained_variance
weighted_loadings = weighted_loadings / np.sum(weighted_loadings)
weights_df2 = pd.DataFrame({'지표': X.columns, '가중치(PCA)': weighted_loadings})

print(weights_df2)


         지표  가중치(PCA)
0    CO2배출량  0.134105
1    CO2증감률  0.160921
2    보호구역비율  0.138432
3      산림면적  0.162345
4    환경규제강도  0.136352
5  NDGAIN지수  0.125571
6   재생에너지비율  0.142274


In [18]:
weights_df2.to_csv('환경가치지수_세부지표별_PCA가중치.csv', index=False)

from google.colab import files
files.download('환경가치지수_세부지표별_PCA가중치.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
# weights_df2가 '지표' 컬럼과 '가중치(PCA)' 컬럼을 갖고 있다고 가정
# 이미 앞에서 만든 경우 이 부분은 생략 가능


In [14]:
# X_scaled : 국가명 제외, 세부지표 표준화 값
# df_merged['국가'] : 국가명


In [15]:
import numpy as np
import pandas as pd

# 가중치 배열
w = weights_df2['가중치(PCA)'].values  # (지표 수, )

# 국가별 표준화 지표 데이터 (2차원 배열, 행:국가, 열:지표)
env_score = X_scaled @ w  # 행렬곱 (각 국가별 환경지수)

# 데이터프레임으로 정리
env_index = pd.DataFrame({
    '국가': df_merged['국가'],
    '환경가치지수': env_score
})

print(env_index)


              국가    환경가치지수
0          Egypt  0.008595
1        Morocco  0.253971
2        Algeria -0.127177
3        Tunisia  0.088857
4        Nigeria  0.304636
5          Ghana  0.481293
6        Senegal -0.153028
7    Ivory Coast -0.409360
8           Mali  0.208426
9   Burkina Faso  0.152437
10         Benin -0.213722
11         Kenya -0.318673
12      Tanzania -0.109182
13        Uganda  0.188014
14        Rwanda  0.057251
15    Madagascar  0.205183
16      Cameroon  0.101060
17         Gabon -0.595106
18         Congo -0.063390
19  South Africa  0.337971
20        Angola -0.088503
21        Zambia  0.154604
22      Botswana  0.281309
23    Mozambique -0.745468


In [16]:
env_index.to_csv('아프리카24개국_환경가치지수_PCA.csv', index=False, encoding='utf-8-sig')


In [17]:
from google.colab import files
files.download('아프리카24개국_환경가치지수_PCA.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [19]:
weights_df2.to_csv('환경가치지수_세부지표별_PCA가중치.csv', index=False)

from google.colab import files
files.download('환경가치지수_세부지표별_PCA가중치.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
env_index.to_csv('환경가치지수_세부지표별_PCA가중치.csv', index=False, encoding='utf-8-sig')

In [21]:
weights_df2.to_csv('환경가치지수_세부지표별_PCA가중치.csv', index=False)

from google.colab import files
files.download('환경가치지수_세부지표별_PCA가중치.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
weights_df2.to_csv('환경가치지수_세부지표별_PCA가중치.csv', index=False, encoding='utf-8-sig')


In [23]:
from google.colab import files
files.download('환경가치지수_세부지표별_PCA가중치.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [24]:
# 상위 5개만 미리보기
print(weights_df2.head())

# 혹은 더 보기
print(weights_df2)

# Colab에서는 그냥 변수명만 입력해도 표로 예쁘게 나옴
weights_df2.head()   # 또는
weights_df2          # 전체 다 보고 싶으면


       지표  가중치(PCA)
0  CO2배출량  0.134105
1  CO2증감률  0.160921
2  보호구역비율  0.138432
3    산림면적  0.162345
4  환경규제강도  0.136352
         지표  가중치(PCA)
0    CO2배출량  0.134105
1    CO2증감률  0.160921
2    보호구역비율  0.138432
3      산림면적  0.162345
4    환경규제강도  0.136352
5  NDGAIN지수  0.125571
6   재생에너지비율  0.142274


,지표,가중치(PCA)
0,CO2배출량,0.134105
1,CO2증감률,0.160921
2,보호구역비율,0.138432
3,산림면적,0.162345
4,환경규제강도,0.136352
5,NDGAIN지수,0.125571
6,재생에너지비율,0.142274


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51473 (\N{HANGUL SYLLABLE JUNG}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52824 (\N{HANGUL SYLLABLE CI}) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
_df_1['가중치(PCA)'].plot(kind='hist', bins=20, title='가중치(PCA)')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
_df_7['가중치(PCA)'].plot(kind='line', figsize=(8, 4), title='가중치(PCA)')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 48176 (\N{HANGUL SYLLABLE BAE}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52636 (\N{HANGUL SYLLABLE CUL}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 47049 (\N{HANGUL SYLLABLE RYANG}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51613 (\N{HANGUL SYLLABLE JEUNG}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_8['지표'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_8, x='index', y='지표', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51473 (\N{HANGUL SYLLABLE JUNG}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52824 (\N{HANGUL SYLLABLE CI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 48176 (\N{HANGUL SYLLABLE BAE}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52636 (

from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_9['지표'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_9, x='가중치(PCA)', y='지표', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [25]:
# 상위 5개만 미리보기
print(env_index.head())

# 혹은 전체
env_index.head(10)   # 상위 10개
env_index            # 전체 표 보기


        국가    환경가치지수
0    Egypt  0.008595
1  Morocco  0.253971
2  Algeria -0.127177
3  Tunisia  0.088857
4  Nigeria  0.304636


,국가,환경가치지수
0,Egypt,0.008595
1,Morocco,0.253971
2,Algeria,-0.127177
3,Tunisia,0.088857
4,Nigeria,0.304636
5,Ghana,0.481293
6,Senegal,-0.153028
7,Ivory Coast,-0.409360
8,Mali,0.208426
9,Burkina Faso,0.152437


from matplotlib import pyplot as plt
_df_10['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 54872 (\N{HANGUL SYLLABLE HWAN}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44221 (\N{HANGUL SYLLABLE GYEONG}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52824 (\N{HANGUL SYLLABLE CI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51648 (\N{HANGUL SYLLABLE JI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 49688 (\N{HANGUL SYLLABL

from matplotlib import pyplot as plt
_df_11['환경가치지수'].plot(kind='hist', bins=20, title='환경가치지수')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['환경가치지수']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_13.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('환경가치지수')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_14.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_15['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 54872 (\N{HANGUL SYLLABLE HWAN}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44221 (\N{HANGUL SYLLABLE GYEONG}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 44032 (\N{HANGUL SYLLABLE GA}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 52824 (\N{HANGUL SYLLABLE CI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 51648 (\N{HANGUL SYLLABLE JI}) missing from font(s) DejaVu Sans.
  plt.savefig(
/usr/local/lib/python3.11/dist-packages/google/colab/_quickchart_lib.py:32: UserWarning: Glyph 49688 (\N{HANGUL SYLLABL

from matplotlib import pyplot as plt
_df_16['환경가치지수'].plot(kind='line', figsize=(8, 4), title='환경가치지수')
plt.gca().spines[['top', 'right']].set_visible(False)